In [ ]:
%pip install nilearn
%pip install nltools
%pip install git+https://github.com/ContextLab/timecorr@9d567e2

In [1]:
import nibabel as nib
import nilearn as nl
import nltools as nlt

In [2]:
import os
import pandas as pd
from matplotlib.colors import ListedColormap
DATA_PATH = "./data"
parcellation_key_path = os.path.join(DATA_PATH, "Schaefer2018_1000Parcels_17Networks_order.txt")
import os
import pandas as pd
from matplotlib.colors import ListedColormap

DATA_PATH = "./data"
parcellation_key_path = os.path.join(DATA_PATH, "Schaefer2018_1000Parcels_17Networks_order.txt")


def read_parcellation_file(path):
    return pd.read_csv(path, sep='\t', header=None, names=['id', 'name', 'r', 'g', 'b', 't']).drop(columns='t')


def parse_parcellation_metadata(df):
    name_parts = df['name'].str.split('_', expand=True)
    df['study'] = name_parts[0]
    df['hemisphere'] = name_parts[1].str[0]
    df['network_raw'] = name_parts[2]
    return df


def map_network_labels(df):
    lookup_table = {
        'VisCent': 'Visual central',
        'VisPeri': 'Visual peripheral',
        'SomMotA': 'Somatomotor A',
        'SomMotB': 'Somatomotor B',
        'DorsAttnA': 'Dorsal attention A',
        'DorsAttnB': 'Dorsal attention B',
        'SalVentAttnA': 'Salience/ventral attention A',
        'SalVentAttnB': 'Salience/ventral attention B',
        'LimbicA': 'Limbic A',
        'LimbicB': 'Limbic B',
        'ContA': 'Control A',
        'ContB': 'Control B',
        'ContC': 'Control C',
        'DefaultA': 'Default mode A',
        'DefaultB': 'Default mode B',
        'DefaultC': 'Default mode C',
        'TempPar': 'Temporal parietal',
    }

    df['network'] = df['network_raw'].map(lookup_table)
    network_names = list(lookup_table.values())
    network_codes = {name: i + 1 for i, name in enumerate(network_names)}
    df['code'] = df['network'].map(network_codes)
    return df


def finalize_key(df):
    df.drop(columns=['name', 'network_raw'], inplace=True)
    df.set_index('id', inplace=True)
    df.loc[0, 'code'] = 0  # background code
    return df


def build_network_colormap():
    network_colors = {
        'Visual central': '#BE1E2D',
        'Visual peripheral': '#C85248',
        'Somatomotor A': '#F15A29',
        'Somatomotor B': '#F47C4B',
        'Dorsal attention A': '#F7941D',
        'Dorsal attention B': '#FAA74A',
        'Salience/ventral attention A': '#F9ED32',
        'Salience/ventral attention B': '#FBF392',
        'Limbic A': '#009444',
        'Limbic B': '#7CB380',
        'Control A': '#1C75BC',
        'Control B': '#7D9FD3',
        'Control C': '#556683',
        'Default mode A': '#92278F',
        'Default mode B': '#B37AB4',
        'Default mode C': '#844984',
        'Temporal parietal': '#DA1C5C',
    }

    colors = list(network_colors.values())
    return ListedColormap(colors, N=len(colors) * 2, name='networks'), network_colors


def build_parcellation_key(key_path):
    df = read_parcellation_file(key_path)
    df = parse_parcellation_metadata(df)
    df = map_network_labels(df)
    df = finalize_key(df)
    cmap, color_dict = build_network_colormap()
    return df, cmap, color_dict


In [3]:
df, cmap, color_dict = build_parcellation_key(parcellation_key_path)

In [4]:
df

,r,g,b,study,hemisphere,network,code
id,,,,,,,
1,120.0,18.0,135.0,17Networks,L,Visual central,1.0
2,120.0,18.0,136.0,17Networks,L,Visual central,1.0
3,120.0,18.0,137.0,17Networks,L,Visual central,1.0
4,120.0,18.0,138.0,17Networks,L,Visual central,1.0
5,120.0,17.0,130.0,17Networks,L,Visual central,1.0
...,...,...,...,...,...,...,...
997,9.0,42.0,250.0,17Networks,R,Temporal parietal,17.0
998,9.0,42.0,251.0,17Networks,R,Temporal parietal,17.0
999,9.0,42.0,252.0,17Networks,R,Temporal parietal,17.0


In [6]:
color_dict

{'Visual central': '#BE1E2D',
 'Visual peripheral': '#C85248',
 'Somatomotor A': '#F15A29',
 'Somatomotor B': '#F47C4B',
 'Dorsal attention A': '#F7941D',
 'Dorsal attention B': '#FAA74A',
 'Salience/ventral attention A': '#F9ED32',
 'Salience/ventral attention B': '#FBF392',
 'Limbic A': '#009444',
 'Limbic B': '#7CB380',
 'Control A': '#1C75BC',
 'Control B': '#7D9FD3',
 'Control C': '#556683',
 'Default mode A': '#92278F',
 'Default mode B': '#B37AB4',
 'Default mode C': '#844984',
 'Temporal parietal': '#DA1C5C'}